# Human In The Loop Patterns

Humans may be needed:

1. Validation of a workflow execution
2. Assistance for executing a specific step in a workflow
3. Intervention/modification of a step result

They can be implemented as steps within a workflow.

In [ ]:
from typing import Any, Awaitable, Protocol, runtime_checkable

from llama_index.core.workflow import (
    Context,
    StartEvent,
    StopEvent,
    Workflow,
    step,
)
from llama_index.llms.openai import OpenAI
from llama_index.core.bridge.pydantic import BaseModel, Field

In [ ]:
class IncrementAdventureEvent(Event):
    new_plot: str
    new_actions: List[str]


class HumanInputEvent(Event):
    chosen_action: str

In [ ]:
"""
CHOOSE YOUR OWN ADVENTURE TEMPLATE

PLOT: ...
ACTIONS: ...
CHOICE: ...

PLOT: ...
ACTIONS: ...
CHOICE: ...

FINAL_PLOT: ...
"""

In [ ]:
class ChooseYourOwnAdventureWorkflow(Workflow):
    def __init__(self, max_steps: int = 3):
        self.llm = OpenAI("gpt-4o")
        self.max_steps = 3

    @step
    async def advance_story(
        self, ctx: Context, ev: StartEvent | HumanInputEvent
    ) -> IncrementAdventureEvent | StopEvent:
        blocks = ctx.get("blocks", [])

        if isinstance(ev, StartEvent):
            # start a new story
            new_plot, new_actions = self.llm.predict
        else:
            chosen_action = str(ev.get("chosen_action"))
            # continue story with chosen action
            new_plot, new_actions = self.llm.predict

        return IncrementAdventureEvent(
            new_plot=new_plot, new_actions=new_actions
        )

    @step
    async def prompt_human(
        self, ctx: Context, ev: IncrementAdventureEvent
    ) -> HumanInputEvent:
        num_steps = ctx.get("num_steps")

        # get human input
        human_prompt = ...
        human_input = input(human_prompt)

        # clean up selection

Or as a `Workflow` themselves:

In [ ]:
class HumanInputWorkflow(Workflow):
    @step
    async def human_input(self, ev: StartEvent) -> StopEvent:
        prompt = str(ev.get("prompt", ""))
        human_input = input(prompt)
        return StopEvent(result=human_input)